In [ ]:
import msprime, tskit
import matplotlib.pyplot as plt # Only needed for plotting
import numpy as np
import pandas as pd
import random
from IPython.display import SVG
from matplotlib.patches import Polygon

# Analyses with tskit

At the end of the previous worksheet, we learnt how to convert a tree sequence's mutational information back into a VCF sequence file. This allows use to use software like scikit-allel, plink, bcftools and vcftools on our simulated dataset.

However, you may not need to do this. There's a lot you can do entirely within `tskit`, and these native methods are often quicker and more efficient than methods that work directly on the sequence data.

 - 4.1: Site statistics
 - 4.2: One-way and multi-way statistics
 - 4.3: Simplify
 - 4.4: Ancestry: identity-by-descent
 - 4.4: Ancestry: population-based ancestry
 - 4.5: Branch statistics

###  A basic simulated tree sequence to use

First, we'll simulate a tree sequence dataset with mutations to use in the rest of this notebook.
Here's a simple 3-population model, where the 2 pops split 50 generations ago from an ancestral population, and the third is formed after an admixture event between these two. One of these modern-day unadmixed pops is smaller than the other. 

In [ ]:
demography = msprime.Demography()
demography.add_population(name="SMALL", initial_size=200)
demography.add_population(name="BIG", initial_size=500)
demography.add_population(name="ADMIX", initial_size=200)
demography.add_population(name="ANC", initial_size=500)
demography.add_admixture(
    time=20, derived="ADMIX", ancestral=["SMALL", "BIG"], proportions=[0.5, 0.5])
demography.add_population_split(time=600, derived=["SMALL", "BIG"], ancestral="ANC")
# demography.debug()

In [ ]:
ts = msprime.sim_ancestry(samples={"SMALL": 10, "BIG": 10, "ADMIX" : 10},
                          demography=demography, random_seed=2432,
                         sequence_length=5e7, recombination_rate=1e-8)
ts = msprime.sim_mutations(ts, rate=1e-8, random_seed=6151)
ts

In [ ]:
tss = ts.simplify(samples=[1, 11, 21, 31, 41, 51])
mts1 = msprime.sim_mutations(tss, rate=1e-8, random_seed=1812)
mts2 = msprime.sim_mutations(tss, rate=1e-8, random_seed=1901)

## 4.1 Site statistics

(3.40pm)

Suppose you wanted to calculate something like mean pairwise diversity with some sequence data of $n$ samples.
Here's an equation you can use:

$$ \pi = \dfrac{1}{n(n-1)/2}\sum_{i=1}^{n-1} \sum_{j=i+1}^n k_{ij} $$

where $k_{ij}$ is the number of sites where sequences $i$ and $j$ have a different allele. (Got this from Matt Hahn's MolPopGen book.)

Consider how you would calculate this with sequence data if you had to do this by hand. Suppose you have $n$ sequenes typed at $m$ different sites.

```
   ...GTAACGCGATAAGAGATTAGCCCAAAAACACAGACATGGAAATAGCGTA...
   ...GTAACGCGATAAGAGATTAGCCCAAAAACACAGACATGGAAATAGCGTA...
   ...GTAACGCGATAAGATATTAGCCCAAAAACACAGACATGGAAATAGCGTA...
   ...GTAACGCGATAAGATATTAGCCCAAAAACACAGACATGGAAATAGCGTA...
   ...GTAACGCGATAAGATATTAGCCCAAAAACACAGACATGGAAATAGCGTA...
   ...GTAACGCGATAAGATATTAGCCCAAAAACACAGACATGGTAATAGCGTA...
   ...GTAACGCGATAAGATATTAGCCCAAAAACACAGACATGGTAATAGCGTA...
```

First you would look at the first two alleles in the first two sequences -- are they the same or different? Then you would move to the next position -- are they the same or different? To perform the full calculation, you would repeat this operation at each site, over each possible pair of sequences. The scaling of this procedure is

$$ O\left( n^2 m \right) $$

ie. quadratic in the number of samples $n$, and linear in the number of sites $m$.

(Would be good to have a tree sequence picture somewhere here -- look through the paper again.)

However, these sequences are simply a consequence of *mutations* on the tree sequence. 
We've already learnt that recording these mutations directly allows us to *store* these sequences in a more compact way -- it turns out that they also allow you to calculate statistics on these sequences more efficiently too. The basic idea is that you place weights on each of the sample nodes in the tree and then 'propagate' these upwards through the tree. (Is this too vague to be helpful?)

This is an example of what `tskit` classifies as a *site statistic*. Essentially, this means... (find a proper definition somewhere: something to do with counts of alleles? A summary of the allele frequency spectrum?).

See this paper for further details, especially if you are interested in coming up with tree sequence statistics of your own. (In theory, it should be possible to calculate any statistic that summarises the allele frequency spectrum using the framework described in this paper).

Peter Ralph, Kevin Thornton, Jerome Kelleher, Efficiently Summarizing Relationships in Large Samples: A General Duality Between Statistics of Genealogies and Genomes, Genetics, Volume 215, Issue 3, 1 July 2020, Pages 779–797, https://doi.org/10.1534/genetics.120.303253

### The basic syntax: (nucleotide diversity)

(3.45pm)

`tskit` uses very similar syntax for all of its inbuilt statistics, so we'll explore the options using `diversity()` as a case study.


In [ ]:
%%time
ts.diversity()

Notice how much faster this calculation runs compared with the one we did using `allel` at the end of the previous notebook (update this!!!).

By default, `tskit` presents a normalised version of the statistic  scaled by the length of the region represented in `ts`. This allows you to make comparisons between different tree sequences that may be of different lengths. However, this isn't how all other genetic software computes diversity -- if you wish to disable `tskit`'s default behaviour, use the `span_normalise` argument.

In [ ]:
div = ts.diversity(span_normalise=False)
div[()]

### Calculating statistics on subsets of the samples

(3.50pm)

What else might you want to do? Well, remember our dataset consists of samples from two populations here, of different sizes. We’d expect them to each have different diversity levels, and for these to differ from the overall (sample-wide) diversity rate. We can get this information out by specifying each of these with the `sample_nodes` argument.

A quick and simple way to get all of the sample node IDs from a particular population is to use the `samples()` method. For instance, the following code returns a numpy array holding all of the sample node IDs from 'population 0':

In [ ]:
ts.samples(0)

By cross-checking against the population table, we can see the population IDs of all the populations in our simulated demography.

In [ ]:
ts.tables.populations

Suppose we wanted to calculate the value of the diversity statistic for each of the three contemporary populations in our dataset ('SMALL', 'BIG' and 'ADMIX'), as well as for the pooled set of samples:

In [ ]:
samples_of_interest=[ts.samples(population=0),
                          ts.samples(population=1),
                          ts.samples(population=2),
                          ts.samples()]
ts.diversity(sample_sets=samples_of_interest)

The output is a 1-dimensional numpy array, where each element is a diversity statistic value for one of the sample sets we specified.
Nucleotide diversity is lowest in the set of samples from the small population, and largest in the pooled set of samples, as you'd expect.

Note that you can use any old list of node IDs as inputs to `sample_sets`. This may be useful if your samples of interest correspond to something else (for instance, samples that hold some phenotype of interest).

In [ ]:
ts.diversity(sample_sets=[[12, 14, 18, 7, 8],
                          [23, 19, 10]])

### Genome scans

(4pm)

So far, we’ve just been calculating statistics summarising diversity values along the entire simulated genome. However, in many cases, we might be more interested in how diversity varies along the genome. We can do this using the `windows` argument.

We specify the start and end points of the sequence, and the locations of the breakpoints between each window. For instance, suppose we wanted to specify some windows of length 1Mb covering our 50Mb chromosome:

In [ ]:
breakpoints = [i*1e6 for i in range(0, 50 + 1)]
print(breakpoints)

In [ ]:
div = ts.diversity(sample_sets=samples_of_interest, windows=breakpoints)
print("Dimensions of the output:", div.shape, "\n")
print("Diversity values over the first 10 windows in each sample set:")
print(div[:10,:])

The output is a 50 by 4 array, one row for each of our specified windows. Each element of the row holds diversity value in some particular window amongst one of our sample sets. Let’s plot these:

In [ ]:
names_to_plot = ['SMALL', 'BIG', 'ADMIX', 'ALL']
lines = plt.plot(breakpoints[:-1], div)
plt.legend(lines, names_to_plot);

Because we ran a neutral simulation, we don’t expect to see any significant changes in diversity along the genome, and the variation we do see is likely just noise. Soon, we’ll see another way to clarify this. 

### Multi-way statistics

(4.10pm)

So far we’ve been looking only at diversity, to help us get used to the general syntax of tskit’s statistics interface. But there are many others which work in exactly the same way.

Diversity is an example of a 'one-way' statistic. All of the samples are exchangeable in the calculation: if we were to re-label one of the samples with another label, we’d still get the same statistic.

Other examples of one-way statistics are:

 - [allele_frequency_spectrum](https://tskit.dev/tskit/docs/stable/python-api.html#tskit.TreeSequence.allele_frequency_spectrum)
 - [segregating_sites](https://tskit.dev/tskit/docs/stable/python-api.html#tskit.TreeSequence.segregating_sites)
 - [Tajimas_D](https://tskit.dev/tskit/docs/stable/python-api.html#tskit.TreeSequence.Tajimas_D)
 - [trait_covariance](https://tskit.dev/tskit/docs/stable/python-api.html#tskit.TreeSequence.trait_covariance), [trait_correlation](https://tskit.dev/tskit/docs/stable/python-api.html#tskit.TreeSequence.trait_correlation) and [trait_linear_model](https://tskit.dev/tskit/docs/stable/python-api.html#tskit.TreeSequence.trait_linear_model)
 
These last 3 are interesting ones we'll look at later link [here](https://tskit.dev/tskit/docs/stable/stats.html#sec-stats-notes-trait). 
(Note: it's weird that these last three wouldn't be multi-way: they aren't completely statistically exchangeable for each other, right? Change the way you describe this. Also, these last 3 aren't strictly summaries of the allele frequency spectrum either.)

This is *not* the case for certain other population genetic statistics that we call ‘multi-way statistics’.
In these statistics, you’ll need to pay special heed to the `sample_sets` argument — the online docs will specify exactly how many sample sets are required, and what should supplied in each field, to get the statistic we are interested in.

Consider Patterson's $f_3$ statistic. Looking at this function's entry in the docs (provide link), we'll need to supply three `sample_sets`, with the first of these acting as the 'focal' population. 

In [ ]:
f3_012 = ts.f3(sample_sets=[ts.samples(0), ts.samples(1), ts.samples(2)],
               windows=breakpoints)
f3_102 = ts.f3(sample_sets=[ts.samples(1), ts.samples(0), ts.samples(2)],
               windows=breakpoints)
f3_201 = ts.f3(sample_sets=[ts.samples(2), ts.samples(0), ts.samples(1)],
               windows=breakpoints)

In [ ]:
names_to_plot = ['SMALL;BIG,ADMIX', 'BIG;SMALL,ADMIX', 'ADMIX;BIG,SMALL']
lines = plt.plot(breakpoints[:-1], np.transpose([f3_012, f3_102, f3_201]))
plt.legend(lines, names_to_plot)

The green line `f3(ADMIX;BIG,SMALL)` is consistently below 0. This suggests that `ADMIX` might be an admixed population with gene flow from `BIG` and `SMALL` (which is indeed the case, since this is what we simulated).

Here are some other examples of multi-way statistics provided by tskit: (add in links later)

 - divergence
 - genetic_relatedness
 - Patterson's f2, f3, f4
 - genealogical_nearest_neighbours
 - Fst

*Exercise*. Modify the following code to compute $f_2$ statistics in 5Mb windows along the genome between the samples from populations SMALL and BIG, as well as between populations SMALL and ADMIX.

In [ ]:
breakpoints = [i*1e6 for i in range(0, 50 + 1)]
# print("Breakpoints:", breakpoints)
f2_small_big = ts.f3(sample_sets=[ts.samples(0),
                                  ts.samples(1),
                                  ts.samples(2)],
               windows=breakpoints)
f2_small_admix = ts.f3(sample_sets=[ts.samples(2),
                                  ts.samples(1),
                                  ts.samples(0)],
               windows=breakpoints)

# Plot.
names_to_plot = ['f2(SMALL,BIG)', 'f2(SMALL,ADMIX)']
lines = plt.plot(breakpoints[:-1], np.transpose([f2_small_big, f2_small_admix]))
plt.legend(lines, names_to_plot)


## Trait-based statistics

(4.20pm)

Don't understand output; look up later.

In [ ]:
beak_size = np.array([[np.log10(x+1) for x in range(ts.num_samples)]])
beak_size.shape
# print(beak_size)
ts.trait_linear_model(np.transpose(beak_size))

## Simplify

(4.25pm)

Here are two of the most common ’subset’-type operations we want to do with genomic datasets:

-  Look at the data for a subset of the samples in the original dataset.
- Look at sequence/variant information at only the sites that vary with respect to that subset.

You’ve may have used software like bcftoolls, vcftools to do some combo of these things before if you’ve worked with real data.
`simplify` is the tree sequence version of these operations.

### The basic syntax

’simplify’ requires a list of sample IDs that you wish to include in the new, smaller tree sequence.
Here, we return the tree sequence for all of the samples from population 'ADMIX' (which has a population label of `2`):

In [ ]:
tss = ts.simplify(ts.samples(2))
tss

In [ ]:
ts

We now have a smaller tree sequence holding just those 20 sample chromosomes from population 'ADMIX'.
Note that the number of sites and mutations is also reduced. This is because `simplify` has also removed all the mutations on the edges that were pruned away. Thes alleles arising from these mutations were only inherited by other samples, or are ancestral to everyone in this smaller subset, so they are not variant sites and are arguably irrelevant. (Option to keep them?)

Note also that although there are fewer edges, nodes and mutations in this newer, simplified tree sequence, it's not a drastic difference. This shows the sub-linearity and efficiency of tree sequence structures. 

It turns out that operations like `simplify` are essential using tree sequences in forward-time simulations — see [simplify paper] if you are interested.

## local ancestry

Simulate with a census event

In [ ]:
demography = msprime.Demography()
demography.add_population(name="SMALL", initial_size=200)
demography.add_population(name="BIG", initial_size=500)
demography.add_population(name="ADMIX", initial_size=200)
demography.add_population(name="ANC", initial_size=500)
demography.add_admixture(
    time=20, derived="ADMIX", ancestral=["SMALL", "BIG"], proportions=[0.5, 0.5])
demography.add_census(time=21)
demography.add_population_split(time=600, derived=["SMALL", "BIG"], ancestral="ANC")
# demography.debug()

In [ ]:
ts = msprime.sim_ancestry(samples={"SMALL": 10, "BIG": 10, "ADMIX" : 10},
                          demography=demography, random_seed=1001,
                         sequence_length=5e7, recombination_rate=1e-8)
ts

Get census nodes

In [ ]:
census_nodes = []
for n in ts.nodes():
    if n.time == 21:
        census_nodes.append(n.id)

Apply link-ancestors

In [ ]:
local_ancestry = ts.tables.link_ancestors(samples=[40,41], ancestors=census_nodes)

In [ ]:
local_ancestry

Note that every number in the 'parent' table above is one of the nodes in `ancestors_1000_gens`. This is essentially what `link_ancestors` does -- it shows you directly which of your samples descend from which ancestors.

3. Replace the parent nodes with their population label

In [ ]:
def get_population_id(node, ts):
    return ts.tables.nodes.population[node]

In [ ]:
ancestry_table = pd.DataFrame(
    data = {
        'left': local_ancestry.left,
        'right': local_ancestry.right,
        'population' : [get_population_id(u, ts) for u in local_ancestry.parent],
        'child' : local_ancestry.child
    }
)

print(ancestry_table)

If you want: sort, and squash

In [ ]:
# import sys
# !{sys.executable} -m pip install pandas --upgrade

In [ ]:
ancestry_table.sort_values(by=['child','left'], inplace=True, ignore_index=True)

ancestry_table

In [ ]:
new_sample = []
new_left = []
new_right = []
new_population = []

for ind, row in ancestry_table.iterrows():
    if ind > 0 and row['left']==new_right[-1] and row['population'] == new_population[-1] and row['child'] == new_sample[-1]:
        new_right[-1] = row['right']
    else:
        new_sample.append(row['child'])
        new_left.append(row['left'])
        new_right.append(row['right'])
        new_population.append(row['population'])
        
squashed_ancestry_table = pd.DataFrame({
    'child': [int(i) for i in new_sample],
    'left' : new_left,
    'right': new_right,
    'population' : [int(p) for p in new_population]
})

squashed_ancestry_table

In [ ]:
# def plot_ancestry_chunk(row, chrom):
#     l = row.left*1e-6
#     r = row.right*1e-6
#     p = row.population
#     if int(p) == 0:
#         c = 'blue'
#     elif int(p) == 1:
#         c = 'red'
#     print('p is', p)
#     chunk = np.array([[l, 0], [r, 0], [r, 1], [l, 1]])
#     chrom.add_patch(Polygon(xy=chunk, color = c))

In [ ]:
chrom_labels = {40: 'chr0', 41: 'chr1'}
colors = ['red', 'blue']
length = ts.sequence_length

fig, (chr0, chr1) = plt.subplots(2, figsize=(10,2))
fig.suptitle('Ancestry in admixed individual')
fig.frameon=False
fig.legend(
    handles = [Polygon(xy = np.array([[0,0],[0,1],[1,1],[1,0]]), color = 'blue'),
              Polygon(xy = np.array([[0,0],[0,1],[1,1],[1,0]]), color = 'red')],
    labels = ['SMALL', 'BIG'],
    loc = 'right'
)
for ind, row in squashed_ancestry_table.iterrows():
    chunk = np.array([[row.left/length, 0], [row.right/length, 0],
                      [row.right/length, 1], [row.left/length, 1]])
    if chrom_labels[row.child] == 'chr0':
        chr0.add_patch(Polygon(xy=chunk, color = colors[int(row.population)]))
    elif chrom_labels[row.child] == 'chr1':
        chr1.add_patch(Polygon(xy=chunk, color = colors[int(row.population)]))

chr0.set_ylabel('Chrom 0')
chr0.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
chr1.set_xticks(ticks= [0.2, 0.4, 0.6, 0.8, 1.0], labels=[10, 20, 30, 40, 50])
chr1.set_xlabel('Chromosomal position (Mb)')
chr1.set_ylabel('Chrom 1')
chr1.tick_params(left=False, labelleft=False)
    

## ibd-segments

In [ ]:
ibd_info = ts.ibd_segments()
print(ibd_info)

In [ ]:
ibd_info = ts.ibd_segments(
    within = [1, 2, 3, 4],
    store_segments=True
)
print(ibd_info)

In [ ]:
# def print_ibd(find_ibd_output):
#     for key in find_ibd_output.keys():
#         print('Common ancestral segments for sample pair', key, ':')
#         ibd = find_ibd_output[key]
#         for i in range(len(ibd.left)):
#             print('Interval: [', ibd.left[i], ibd.right[i], '),', 
#                  'ancestral node:', ibd.node[i])
#         print()
        
# print(ibd_info)

In [ ]:
ibd_info = ts.ibd_segments(
    between = [[1, 2], [3, 4]],
    store_segments=True
)

print(ibd_info)

In [ ]:
ibd_info = ts.ibd_segments(
    within = [1, 2, 3, 4],
    min_span = 2e6,
    store_segments=True
)
print(ibd_info)

In [ ]:
ibd_info = ts.ibd_segments(
    within = [0, 1, 2, 3],
    max_time = 20,
    store_segments=True
)
print(ibd_info)

Here's an IBD version of the plot we made before:

### Branch stats

We saw, and briefly noted, how apparently the statistics were along the genome. As you know by now, some of this is just noise, and it can be difficult doing this statistic test by just eyeballing the data. A big part of many population genetic tests, including tests for selection, introgressed regions, is being able to make educated guesses about where the patterns you see are *not* due to this.
This is complicated by the fact that there are several different types of randomness in genetic models that interact with each other.
There is randomness in the genealogical trees that are produced. If you have two populations same in size, still may have coalescences (common ancestors) in slightly different times (expect those to obey the same distribution), each simulation is just one realisation of potential history, and randomness of recombination means locations, lengths along which those relationships hold also random (this is what sim_ancestry models).

On top of that, there’s added randomness due to randomness of mutation processes. Even if you have two regions of a genome in a given individual that has been inherited from the same two regions of some ancestor, inherited via the same set of genealogical descendants, there may be a different density of mutations simply because of the randomness of mutations.

Consider how different one of our trees looks...

In [ ]:
# [Look at two different trees that are very far apart from each other]
sts = ts.simplify(samples=[1, 11, 21, 31, 41, 51])
tree = sts.first()
display(SVG(tree.draw(width=300)))

from one much further along the genome

In [ ]:
tree = sts.last()
display(SVG(tree.draw(width=300)))

When you calculate statistics based on allele-frequencies — what we call site-based statistics — *both* of these sources of noise contribute to the noisiness you see.

In [ ]:
# [Look at the same tree, overlaid with different mutations each time]
display(SVG(mts1.last().draw(width=300)))

In [ ]:
display(SVG(mts2.last().draw(width=300)))

But in tree sequences, you have information about branches which allows you to bypass this latter type of mutation.
Instead of moving up the trees and updating counts of some statistic every time you come across a mutation, update the statistic based on the *number of mutations you expect*. This is proportional to the branch length (since we’re assuming uniformity of mutations over time). This is the basic idea of branch stats — low variance form of site-based statistic that substitutes the length of each branch for the actual number of mutations on that branch. Note this deals with the second type of randomness, so is overall lower variance.

Here are the diversity stats we looked at before, this time with the branch versions included. Notice that the branch and site versions are clearly correlated, which makes sense — the mutations on a given tree are conditional on that tree — but the branch versions have significantly lower variance.

In [ ]:
# [Diversity stat]
div = ts.diversity(sample_sets=samples_of_interest, windows=breakpoints, mode='branch',span_normalise=True)
names_to_plot = ['SMALL', 'BIG', 'ADMIX', 'ALL']
lines = plt.plot(breakpoints[:-1], div)
plt.legend(lines, names_to_plot);

In [ ]:
# [Diversity stat]
breakpoints = [i*1e6 for i in range(0, 50 + 1)]
div_site = ts.diversity(sample_sets=samples_of_interest, windows=breakpoints, mode='site')
div_branch = ts.diversity(sample_sets=samples_of_interest, windows=breakpoints, mode='branch')
names_to_plot = ['SMALL', 'BIG', 'ADMIX', 'ALL']
# lines = plt.plot(breakpoints[:-1], div_site)
lines = plt.plot(breakpoints[:-1], div_site/div_branch)
plt.legend(lines, names_to_plot);

Since publication, has been leveraged in other more specific applications (Charleston Chiang, genetic_relatedness, our work). May be that conditional on having accurate tree , simulated or otherwise, departures between the branch and site versions of a statistic may indicate other processes of interest — mutagens affected particular genomic regions, for instance.

(Is there some way to scale according to the mutation rate?)